In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

batch_size = 32
# Transform to convert images to PyTorch tensors and normalize them
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data for MNIST
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


In [ ]:
all_images = []
all_indices = []

for i, (images, labels) in enumerate(trainloader):
    all_images.extend(images)
    all_indices.extend(range(i * len(images), (i + 1) * len(images)))

# Convert the list to a NumPy array
all_images = torch.stack(all_images)
all_indices = torch.tensor(all_indices)



# Function to display an image given its index
def imshow_by_index(index):
    img = all_images[index] / 2 + 0.5  # Unnormalize
    plt.imshow(img.permute(1, 2, 0))  # Convert from Tensor image
    plt.show()

# Example: Display the first four images
for i in range(4):
    imshow_by_index(i)

In [ ]:
class TrainableNode(nn.Module):
    def __init__(self, input_features, output_features):
        super(TrainableNode, self).__init__()
        self.linear = nn.Linear(input_features, output_features)

    def forward(self, x):
        return self.linear(x)


class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        # Define your layers here
        self.fc1 = nn.Linear(784, 128)
        self.explainable_node = TrainableNode(128, 1)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x, return_explainable=False):
        x = x.view(x.size(0), -1)
        x1 = torch.relu(self.fc1(x))
        if return_explainable:
            explainable_output = self.explainable_node(x1).squeeze()
        x = torch.relu(self.fc2(x1))
        x = self.fc3(x)
        if return_explainable:
            return x, explainable_output
        return x



    def forward_with_explainable(self, x):
        # This method is specifically for when you want to use the explainable node
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        explainable_output = self.explainable_node(x)
        return explainable_output
        

class ExplainableNetwork:
    def __init__(self, model):
        self.model = model
        self.activations_per_input = {}
        self.weights_per_layer = {}
        self.gradients_per_input = {} 
        self.current_epoch = 0
        self.num_epochs = 0
        self.batch_counter = 0

        self.expected_outputs = []
        self.predicted_outputs = []
        
        # Register hooks for each layer
        for name, module in model.named_modules():
            if name == 'explainable_node':  # Skip the explainable_node by its name
                continue
            if isinstance(module, torch.nn.Linear):
                self.weights_per_layer[name] = None
                module.register_forward_hook(self.create_forward_hook(name))
                module.register_full_backward_hook(self.create_backward_hook(name))


    def create_forward_hook(self, layer_name):
        def forward_hook(module, input, output):
            # On the last epoch, store activations for each input
            if self.current_epoch == self.num_epochs - 1:
                self.store_activations(layer_name, output)
            # Always store the current weights
            self.weights_per_layer[layer_name] = module.weight.data.clone()
        return forward_hook

    def create_backward_hook(self, layer_name):
        def backward_hook(module, grad_input, grad_output):
            # Only store gradients on the last epoch
            if self.current_epoch == self.num_epochs - 1:
                # grad_output[0] contains the gradient with respect to the output of the layer
                self.store_gradients(layer_name, grad_output[0])
        return backward_hook


    def store_activations(self, layer_name, output):
        self._store_per_input(self.activations_per_input, layer_name, output)

    def store_gradients(self, layer_name, gradient):
        # Store gradients for the last epoch
        if self.current_epoch == self.num_epochs - 1:
            batch_size = gradient.size(0)
            for i in range(batch_size):
                input_index = self.batch_counter * batch_size + i
                if input_index not in self.gradients_per_input:
                    self.gradients_per_input[input_index] = {}
                self.gradients_per_input[input_index][layer_name] = gradient[i].detach().cpu().numpy()
    
            # Increment the backward batch counter only after processing the first layer (which is the last layer in the forward pass)


    def _store_per_input(self, storage_dict, layer_name, tensor):
        batch_size = tensor.size(0)
        for i in range(batch_size):
            input_index = self.batch_counter * batch_size + i
            if input_index not in storage_dict:
                storage_dict[input_index] = {}
            storage_dict[input_index][layer_name] = tensor[i].detach().cpu().numpy()

        if layer_name == list(self.model.named_modules())[-1][0]:
            self.batch_counter += 1

    def start_epoch(self, epoch, num_epochs):
        self.current_epoch = epoch
        self.num_epochs = num_epochs
        if epoch == num_epochs - 1:
            self.batch_counter = 0  # Reset the batch counter on the last epoch

    def get_activations(self, input_index):
        return self.activations_per_input.get(input_index, None)

    def get_gradients(self, input_index):
        return self.gradients_per_input.get(input_index, None)

    def print_structure(self):
        print("Explainable Network Structure and Data Summary:")

        # Check if there is any data collected
        if not self.activations_per_input:
            print("No activations data collected.")
            return

        if not self.gradients_per_input:
            print("No gradients data collected.")
            return

        # Print information about activations
        print("\nActivations:")
        first_input_activations = next(iter(self.activations_per_input.values()))
        for layer_name, activations in first_input_activations.items():
            num_nodes = len(activations)
            num_data_points = len(self.activations_per_input)
            print(f"Layer {layer_name}: {num_nodes} nodes, {num_data_points} data points each")

        # Print information about gradients
        print("\nGradients:")
        first_input_gradients = next(iter(self.gradients_per_input.values()))
        for layer_name, gradients in first_input_gradients.items():
            num_nodes = len(gradients)
            num_data_points = len(self.gradients_per_input)
            print(f"Layer {layer_name}: {num_nodes} nodes, {num_data_points} data points each")

    # TODO this uses the most recent weights
    def get_node_input(self, layer_name, node_index, image_index):
        if layer_name not in self.weights_per_layer:
            raise ValueError(f"Layer {layer_name} not found in the model")

        # Convert weights to numpy array
        weights = self.weights_per_layer[layer_name].cpu().numpy()

        if layer_name == 'fc1':
            # For the first layer, the input is the raw image data
            # Assuming you have a way to access the raw input images
            pass
        else:
            # For subsequent layers, we use activations from the previous layer
            prev_layer_name = self.get_previous_layer_name(layer_name)
            prev_activations = self.activations_per_input[image_index][prev_layer_name]

            # Ensure the prev_activations is a 1D array for proper dot product
            if prev_activations.ndim > 1:
                prev_activations = prev_activations.flatten()

            # Multiply previous layer's activations by the weights for the specific node
            input_to_node = np.dot(prev_activations, weights[node_index])

        return input_to_node

    def get_pre_sum_node_input(self, layer_name, node_index, image_index):
        if layer_name not in self.weights_per_layer:
            raise ValueError(f"Layer {layer_name} not found in the model")

        # Convert weights to numpy array
        weights = self.weights_per_layer[layer_name].cpu().numpy()

        if layer_name == 'fc1':
            # For the first layer, the input is the raw image data
            # Assuming you have a way to access the raw input images
            pass
        else:
            # For subsequent layers, we use activations from the previous layer
            prev_layer_name = self.get_previous_layer_name(layer_name)
            prev_activations = self.activations_per_input[image_index][prev_layer_name]

            # Ensure the prev_activations is a 1D array for proper multiplication
            if prev_activations.ndim > 1:
                prev_activations = prev_activations.flatten()

            # Multiply previous layer's activations by the weights for the specific node
            pre_sum_input_to_node = prev_activations * weights[node_index]

        return pre_sum_input_to_node
    
    def get_previous_layer_name(self, current_layer_name):
        # Helper function to get the previous layer's name
        layers = list(self.model.named_modules())
        for i, (name, _) in enumerate(layers):
            if name == current_layer_name:
                return layers[i - 1][0] if i > 0 else None

        return None





In [ ]:
model = MNISTNet().to(device)
explainable_net = ExplainableNetwork(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 2  # Adjust as needed

for epoch in range(num_epochs):
    explainable_net.start_epoch(epoch, num_epochs)
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, targets)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        if epoch == num_epochs - 1:
            explainable_net.expected_outputs.extend(targets.cpu().numpy())
            
            # Convert outputs to predicted class labels
            _, predicted = torch.max(outputs, 1)
            explainable_net.predicted_outputs.extend(predicted.cpu().numpy())
        


In [ ]:
explainable_net.print_structure()

In [ ]:
sample_size = 10

print(len(explainable_net.expected_outputs))
print(len(explainable_net.predicted_outputs))

print("Sample of Expected Outputs:", explainable_net.expected_outputs[:sample_size])
print("Sample of Predicted Outputs:", explainable_net.predicted_outputs[:sample_size])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

def plot_scatter_for_correct_predictions(explainable_network, layer_name, node_index):
    activations = []
    gradients = []
    colors = []

    # Define a color map for digits 0-9 using the new syntax
    color_map = matplotlib.colormaps['tab10']

    # Retrieve activations, gradients, and check for correct predictions
    for input_index, (expected, predicted) in enumerate(zip(explainable_network.expected_outputs, explainable_network.predicted_outputs)):
        if True == True:
            activation = explainable_network.get_activations(input_index)
            gradient = explainable_network.get_gradients(input_index)

            if activation is not None and gradient is not None:
                activations.append(activation[layer_name][node_index])
                gradients.append(-gradient[layer_name][node_index])  # Negative of the gradient
                colors.append(color_map(expected))  # Color code based on the correct digit

    # Generate scatter plot
    fig, ax = plt.subplots(figsize=(15,10))  # Explicitly create figure and axes
    scatter = ax.scatter(activations, gradients, c=colors, cmap=color_map, s=15)  # Set dot size with `s`
    ax.set_xlabel('Activation')
    ax.set_ylabel('Negative of Gradient')
    ax.set_title(f'Correct Predictions Scatter Plot for {layer_name} - Node {node_index}')

    # Create a colorbar with the correct axes reference
    cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=color_map, norm=plt.Normalize(0, 9)), ax=ax, ticks=np.arange(0, 10))
    cbar.set_label('Digit')

    ax.grid(True)
    plt.show()

plot_scatter_for_correct_predictions(explainable_net, 'fc2', 1)  # Example for node 5 in the first hidden layer (fc1)



In [ ]:
def plot_scatter_for_extreme_outliers(explainable_network, layer_name, node_index, std_dev_threshold=3):
    activations = []
    gradients = []
    colors = []

    # Define a color map for digits 0-9
    color_map = matplotlib.colormaps['tab10']

    # Retrieve activations, gradients, and check for correct predictions
    for input_index, (expected, predicted) in enumerate(zip(explainable_network.expected_outputs, explainable_network.predicted_outputs)):
        if expected == predicted:
            activation = explainable_network.get_activations(input_index)
            gradient = explainable_network.get_gradients(input_index)

            if activation is not None and gradient is not None:
                activations.append(activation[layer_name][node_index])
                gradients.append(-gradient[layer_name][node_index])  # Negative of the gradient
                colors.append(color_map(expected))  # Color code based on the correct digit

    # Convert lists to numpy arrays for numerical operations
    activations = np.array(activations)
    gradients = np.array(gradients)
    colors = np.array(colors)

    # Calculate mean and standard deviation for activations and gradients
    activation_mean = np.mean(activations)
    activation_std = np.std(activations)
    gradient_mean = np.mean(gradients)
    gradient_std = np.std(gradients)

    # Apply filtering for points that are more than std_dev_threshold away in both dimensions
    extreme_outliers = (np.abs(activations - activation_mean) > std_dev_threshold * activation_std) & \
                       (np.abs(gradients - gradient_mean) > std_dev_threshold * gradient_std)

    # Filter activations, gradients, and colors based on the extreme outlier condition
    activations_extreme = activations[extreme_outliers]
    gradients_extreme = gradients[extreme_outliers]
    colors_extreme = colors[extreme_outliers]

    # Generate scatter plot with extreme outliers
    fig, ax = plt.subplots(figsize=(15, 10))
    scatter = ax.scatter(activations_extreme, gradients_extreme, c=colors_extreme, cmap=color_map, s=15)
    ax.set_xlabel('Activation')
    ax.set_ylabel('Negative of Gradient')
    ax.set_title(f'Extreme Outliers Scatter Plot for {layer_name} - Node {node_index}')

    # Create a colorbar
    cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=color_map, norm=plt.Normalize(0, 9)), ax=ax, ticks=np.arange(0, 10))
    cbar.set_label('Digit')

    ax.grid(True)
    plt.show()

# Example usage:
#plot_scatter_for_extreme_outliers(explainable_net, 'fc2', 3, std_dev_threshold=1.5)  # Adjust std_dev_threshold as needed


In [ ]:
# Normalizes the predictions (i, j) to the total number of predictions(i, _) for all predictions
# Returns a 2D array such that probabilities[i][j] holds the probability of the model
# predicting last_layer[i] as last_layer[j]
def calculate_probabilities(classifier_outcomes, label_counts):
    probabilities = np.zeros_like(classifier_outcomes, dtype=np.float64)
    for i in range(len(label_counts)):
        if label_counts[i] > 0:  # Prevent division by zero
            probabilities[i] = classifier_outcomes[i] / label_counts[i]
    return probabilities


In [ ]:
model.eval()
correct = 0
total = 0

final_layer_size = list(explainable_net.model.named_modules())[-1][1].out_features
# classifier_outcomes[i][j] will store the probability of the model incorrectly classifying digit i as digit j.
# The diagonals store the probability of a correct predicition
classifier_outcomes = np.zeros((final_layer_size, final_layer_size), dtype=np.float64)
label_counts = np.zeros(final_layer_size, dtype=np.int64)

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Initialize classifier_outcomes as the frequency of each prediction (label, pred)
        for label, pred in zip(labels, predicted):
            classifier_outcomes[label.item()][pred.item()] += 1
            # Store the number of total attempts at predicting 'label'
            label_counts[label.item()] += 1

classifier_outcomes = calculate_probabilities(classifier_outcomes, label_counts)
accuracy = 100 * correct / total
print(f'Accuracy of the model on the 10000 test images: {accuracy}%')


In [ ]:
# Prints the contents of classifier_outcomes as a percent or a probability.
# Rounded to four decimal places
def print_classifier_outcomes(classifier_outcomes, mode='percent'):
    size = classifier_outcomes.shape[0]
    for i in range(size):
        for j in range(size):
            if mode == 'percent':
                print(f'a {i} was classified as a {j} {classifier_outcomes[i][j] * 100:.4f}% of the time')
            elif mode == 'probability':
                print(f'The probability of {i} being classified as {j} is {classifier_outcomes[i][j]:.4f}')
print_classifier_outcomes(classifier_outcomes, 'probability')

In [ ]:
# Corrected usage with image_index as an integer
image_index = 0  # Replace with the desired image index
target_layer = 'fc2'
target_node = 1  # Replace with the desired node index

#input_to_node = explainable_net.get_node_input(target_layer, target_node, image_index)



In [ ]:
#print("Input to node {} in layer {} for image index {}:".format(target_node, target_layer, image_index))
#print(input_to_node)



In [ ]:
#pre_sum_input_to_node = explainable_net.get_pre_sum_node_input(target_layer, target_node, image_index)

# Print the result
#print(f"Pre-sum weighted inputs to node {target_node} in layer {target_layer} for image index {image_index}:")
#print(pre_sum_input_to_node)

In [ ]:
def print_activation_for_node(explainable_net, image_index, layer_name, node_index):
    # Check if the requested image index is in the stored activations
    if image_index in explainable_net.activations_per_input:
        image_activations = explainable_net.activations_per_input[image_index]
        
        # Check if the requested layer is in the activations for the image index
        if layer_name in image_activations:
            # Check if the requested node index is within the range of the layer's activations
            if node_index < len(image_activations[layer_name]):
                # Retrieve the specific activation for the node
                activation = image_activations[layer_name][node_index]
                print(f"Activation for image index {image_index}, layer '{layer_name}', node {node_index}: {activation}")
            else:
                print(f"Node index {node_index} is out of range for layer '{layer_name}'.")
        else:
            print(f"Layer '{layer_name}' not found for image index {image_index}. Available layers are: {list(image_activations.keys())}")
    else:
        print(f"No activation information for image index {image_index}. Available image indices are: {list(explainable_net.activations_per_input.keys())}")

# Example usage:
#print_activation_for_node(explainable_net, image_index=0, layer_name='fc1', node_index=1)


In [ ]:
def print_activation_and_weights(explainable_net, layer_name, node_index, image_index):
    # Check if the layer is in the model
    if layer_name not in explainable_net.weights_per_layer:
        raise ValueError(f"Layer {layer_name} not found in the model")

    # Convert weights to numpy array
    weights = explainable_net.weights_per_layer[layer_name].cpu().numpy()

    # Retrieve the activations for the previous layer
    if layer_name == 'fc1':
        # For the first layer, we don't have previous layer activations
        # Assuming you have a way to access the raw input images
        print("Layer 'fc1' does not have previous layer activations.")
    else:
        # For subsequent layers, we use activations from the previous layer
        prev_layer_name = explainable_net.get_previous_layer_name(layer_name)
        prev_activations = explainable_net.activations_per_input[image_index][prev_layer_name]

        # Ensure the prev_activations is a 1D array for proper printing
        if prev_activations.ndim > 1:
            prev_activations = prev_activations.flatten()

        # Get the weights for the specific node
        node_weights = weights[node_index]

        # Print the activations and weights
        print(f"Activations from previous layer '{prev_layer_name}' for image index {image_index}: {prev_activations}")
        print(f"Weights for node {node_index} in layer '{layer_name}': {node_weights}")

# Example usage:
#print_activation_and_weights(explainable_net, layer_name='fc3', node_index=2, image_index=1)


In [ ]:
def DiscriminatedOutputs(explainable_network, positive, trueWithRespectToGradient):
    activations = {}
    extreme_outliers = []
    gradients = {}
    colors = []

    # Define a color map for digits 0-9
    color_map = matplotlib.colormaps['tab10']

    # Retrieve activations, gradients, and check for correct predictions
    for input_index, (expected, predicted) in enumerate(zip(explainable_network.expected_outputs, explainable_network.predicted_outputs)):
        if True == True:
            activation = explainable_network.get_activations(input_index)
            gradient = explainable_network.get_gradients(input_index)

            if activation is not None and gradient is not None:
                activations[input_index] = activation['fc2'][1]
                gradients[input_index] = -gradient['fc2'][1] # Negative of the gradient
                colors.append(color_map(expected))  # Color code based on the correct digit

    # Convert lists to numpy arrays for numerical operations
    activationArray = np.array(list(activations.values()))
    gradientsArray = np.array(list(gradients.values()))
    colors = np.array(colors)

    # Calculate mean and standard deviation for activations and gradients
    activation_mean = np.mean(activationArray)
    activation_std = np.std(activationArray)
    gradient_mean = np.mean(gradientsArray)
    gradient_std = np.std(gradientsArray)

    # Apply filtering for points that are more than std_dev_threshold away in both dimensions
    if positive and trueWithRespectToGradient:
        for keyA, valueA in activations.items():
            if ((valueA - activation_mean) > 1.5 * activation_std) and ((gradients[keyA] - gradient_mean) > 1.5 * gradient_std):
                extreme_outliers.append(keyA)
    elif not positive and not trueWithRespectToGradient:
        for keyA, valueA in activations.items():
            if ((valueA - activation_mean) < 1.5 * activation_std) and ((gradients[keyA] - gradient_mean) < 1.5 * gradient_std):
                extreme_outliers.append(keyA)
                    
    return extreme_outliers
truePos = DiscriminatedOutputs(explainable_net, True, True)
trueNeg = DiscriminatedOutputs(explainable_net, False, True)
print(len(trueNeg))

def display_image_grid(images, rows, cols):
    grid = torchvision.utils.make_grid(images, nrow=cols)
    grid = grid / 2 + 0.5  # Unnormalize
    plt.figure(figsize=(15, 15))  # Adjust the size as needed
    plt.imshow(grid.permute(1, 2, 0))
    plt.show()

# Number of images per grid
images_per_grid = 128  # for example, a 4x4 grid

# Calculate rows and columns for the grid
cols = 10  # Number of columns
rows = images_per_grid // cols

# Display the images in grids
for i in range(0, len(truePos), images_per_grid):
    selected_indices = truePos[i:i+images_per_grid]
    selected_images = torch.stack([all_images[index] for index in selected_indices])
    display_image_grid(selected_images, rows, cols)

In [ ]:
def DiscriminatedOutputs(explainable_network, positive, trueWithRespectToGradient):
    activations = []
    gradients = []
    colors = []

    # Define a color map for digits 0-9
    color_map = matplotlib.colormaps['tab10']

    # Retrieve activations, gradients, and check for correct predictions
    for input_index, (expected, predicted) in enumerate(zip(explainable_network.expected_outputs, explainable_network.predicted_outputs)):
        if expected == predicted:
            activation = explainable_network.get_activations(input_index)
            gradient = explainable_network.get_gradients(input_index)

            if activation is not None and gradient is not None:
                activations.append(activation['fc2'][1])
                gradients.append(-gradient['fc2'][1])  # Negative of the gradient
                colors.append(color_map(expected))  # Color code based on the correct digit

    # Convert lists to numpy arrays for numerical operations
    activations = np.array(activations)
    gradients = np.array(gradients)
    colors = np.array(colors)

    # Calculate mean and standard deviation for activations and gradients
    activation_mean = np.mean(activations)
    activation_std = np.std(activations)
    gradient_mean = np.mean(gradients)
    gradient_std = np.std(gradients)

    # Apply filtering for points that are more than std_dev_threshold away in both dimensions
    extreme_outliers = ((activations - activation_mean) > 1.5 * activation_std) & \
                       ((gradients - gradient_mean) > 1.5 * gradient_std)

    # Filter activations, gradients, and colors based on the extreme outlier condition
    activations_extreme = activations[extreme_outliers]
    gradients_extreme = gradients[extreme_outliers]
    colors_extreme = colors[extreme_outliers]

    print(len(activations_extreme))
    # Generate scatter plot with extreme outliers
    fig, ax = plt.subplots(figsize=(15, 10))
    scatter = ax.scatter(activations_extreme, gradients_extreme, c=colors_extreme, cmap=color_map, s=15)
    ax.set_xlabel('Activation')
    ax.set_ylabel('Negative of Gradient')
    ax.set_title(f'Extreme Outliers Scatter Plot for fc2 - Node 1')

    # Create a colorbar
    cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=color_map, norm=plt.Normalize(0, 9)), ax=ax, ticks=np.arange(0, 10))
    cbar.set_label('Digit')

    ax.grid(True)
    plt.show()
DiscriminatedOutputs(explainable_net, True, True)


In [ ]:

print(np.shape(explainable_net.predicted_outputs))
def visualizePreSumInput(visualizingArray, targetLayer, targetNode, imageIndex):
    pixelColoringMatrix = explainable_net.get_pre_sum_node_input(targetLayer, targetNode, imageIndex)
    # for i in range(len(pixelColoringMatrix)):
    #     if pixelColoringMatrix[i] > 0:
    #         pixelColoringMatrix[i] = 1
    #     else:
    #         pixelColoringMatrix[i] = -1
#visualizePreSumInput(explainable_net, 'fc2', 1, 0)
plot_scatter_for_extreme_outliers(explainable_net, 'fc2', 1, std_dev_threshold=1.5)  # Adjust std_dev_threshold as needed

In [ ]:
images_subset = [all_images[i] for i in truePos]

# Flatten the images
flattened_images = torch.stack(images_subset).view(len(images_subset), -1).numpy()

# Perform PCA to reduce dimensions
pca = PCA(n_components=2)  # reduce to 2 dimensions for visualization
reduced_images = pca.fit_transform(flattened_images)

plt.figure(figsize=(10, 6))
plt.scatter(reduced_images[:, 0], reduced_images[:, 1], s=50)
plt.title("PCA Reduced MNIST Images")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()

In [ ]:
images_subset = torch.stack([all_images[i] for i in truePos])
average_image = images_subset.float().mean(dim=0).squeeze()
plt.imshow(average_image, cmap='gray')
plt.title("Average Image")
plt.colorbar()
plt.show()

In [ ]:
normalized_image = (average_image - average_image.min()) / (average_image.max() - average_image.min())

# Apply a power law transformation to increase contrast, emphasizing the bright pixels
# Squaring the normalized pixel values will make high values brighter and low values dimmer
emphasized_image = normalized_image ** 4

# Plot the emphasized average image
plt.imshow(emphasized_image, cmap='gray')
plt.title("Emphasized Average Image")
plt.colorbar()
plt.show()

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Create dummy data for dataset1 and dataset2
# Let's assume each sample has 784 features (like flattened MNIST images)
dataset1 = torch.rand(100, 784)  # 100 samples from class 0
dataset2 = torch.rand(100, 784)  # 100 samples from class 1


labels1 = torch.zeros(100, dtype=torch.float32)  # Labels for dataset1
labels2 = torch.ones(100, dtype=torch.float32)   # Labels for dataset2

# Combine datasets
combined_dataset = torch.cat([dataset1, dataset2], dim=0)
combined_labels = torch.cat([labels1, labels2], dim=0)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(combined_dataset, combined_labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.explainable_node.parameters(), lr=0.001)

num_epochs = 5  # For demonstration, adjust as needed

for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model.forward_with_explainable(inputs).squeeze()  # Get outputs from the explainable node
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [ ]:
inputs = all_images[1].to(device)

model.eval()

# Forward pass to get both outputs
with torch.no_grad():
    regular_output, explainable_output = model(inputs, return_explainable=True)

print("Regular Output:", regular_output)
print("Explainable Output:", explainable_output)

In [ ]:
import pandas as pd

def plot_extreme_outliers(explainable_network, layer_name, node_index, std_dev_threshold=3, discriminate_classes=[-1,-1], Correct_pred_only=False):
    activations = []
    gradients = []
    colors = []
    labels = []
    classA = discriminate_classes[0]
    classB = discriminate_classes[1]
    
    # Define a color map for digits 0-9
    color_map = matplotlib.colormaps['tab10']

    # Retrieve activations, gradients, and check for correct predictions
    for input_index, (expected, predicted) in enumerate(zip(explainable_network.expected_outputs, explainable_network.predicted_outputs)):
        if expected == predicted or not Correct_pred_only:
            activation = explainable_network.get_activations(input_index)
            gradient = explainable_network.get_gradients(input_index)

            if activation is not None and gradient is not None:
                activations.append(activation[layer_name][node_index])
                gradients.append(-gradient[layer_name][node_index])  # Negative of the gradient
                colors.append(color_map(expected))  # Color code based on the correct digit
                labels.append(expected)
            

    # Convert lists to numpy arrays for numerical operations
    activations = np.array(activations)
    gradients = np.array(gradients)
    labels = np.array(labels)
    colors = np.array(colors)

    # Calculate mean and standard deviation for activations and gradients
    activation_mean = np.mean(activations)
    activation_std = np.std(activations)
    gradient_mean = np.mean(gradients)
    gradient_std = np.std(gradients)

    # Apply filtering for points that are more than std_dev_threshold away in both dimensions
    extreme_outliers = (np.abs(activations - activation_mean) > std_dev_threshold * activation_std) & \
                       (np.abs(gradients - gradient_mean) > std_dev_threshold * gradient_std)

    # Filter activations, gradients, and colors based on the extreme outlier condition
    activations_extreme = activations[extreme_outliers]
    gradients_extreme = gradients[extreme_outliers]
    labels_extreme = labels[extreme_outliers]
    colors_extreme = colors[extreme_outliers]

    # Create Dataframe for easy use
    df = pd.DataFrame({'Activations': activations_extreme, 'Gradients': gradients_extreme, 'Labels': labels_extreme})
    
    if classA > -1 and classB > -1:
        #condition = np.logical_or(colors_extreme[labels == classA], colors_extreme[labels == classB])
        colors_extreme = colors_extreme[df['Labels'].isin([classA, classB])]
        df = df[df['Labels'].isin([classA, classB])]

    # Generate scatter plot with extreme outliers
    fig, ax = plt.subplots(figsize=(15, 10))
    scatter = ax.scatter(df.loc[:, ['Activations']], df.loc[:, ['Gradients']], c= colors_extreme, cmap=color_map, s=15)
    ax.set_xlabel('Activation')
    ax.set_ylabel('Negative of Gradient')
    ax.set_title(f'Extreme Outliers Scatter Plot for {layer_name} - Node {node_index}')

    # Create a colorbar
    cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=color_map, norm=plt.Normalize(0, 9)), ax=ax, ticks=np.arange(0, 10))
    cbar.set_label('Digit')

    ax.grid(True)
    plt.show()


# Example usage:
plot_extreme_outliers(explainable_net, 'fc3', 3, std_dev_threshold=1, Correct_pred_only=False)  # Adjust std_dev_threshold as needed


In [ ]:
import pandas as pd

def plot_extreme_outliers_quad(explainable_network, layer_name, node_index, quadrant, std_dev_threshold=3, selected_class=-1):
    activations = []
    gradients = []
    labels = []
    imgIndx = []
    colors = []
    
     # Define a color map for digits 0-9
    color_map = matplotlib.colormaps['tab10']

    # Retrieve activations, gradients, and check for correct predictions
    for input_index, (expected, predicted) in enumerate(zip(explainable_network.expected_outputs, explainable_network.predicted_outputs)):
        #if expected == predicted: do we want to filter this?
        activation = explainable_network.get_activations(input_index)
        gradient = explainable_network.get_gradients(input_index)

        if activation is not None and gradient is not None:
            activations.append(activation[layer_name][node_index])
            gradients.append(-gradient[layer_name][node_index])  # Negative of the gradient
            labels.append(expected)
            imgIndx.append(input_index)
            colors.append(color_map(expected))  # Color code based on the correct digit
            

    # Convert lists to numpy arrays for numerical operations
    activations = np.array(activations)
    gradients = np.array(gradients)
    labels = np.array(labels)
    imgIndx = np.array(imgIndx)
    colors = np.array(colors)


    # Calculate mean and standard deviation for activations and gradients
    activation_mean = np.mean(activations)
    activation_std = np.std(activations)
    gradient_mean = np.mean(gradients)
    gradient_std = np.std(gradients)

    # Apply filtering for points that are more than std_dev_threshold away in both dimensions
    extreme_outliers = (np.abs(activations - activation_mean) > std_dev_threshold * activation_std) & \
                       (np.abs(gradients - gradient_mean) > std_dev_threshold * gradient_std)

    # Filter activations, gradients, and colors based on the extreme outlier condition
    activations_extreme = activations[extreme_outliers]
    gradients_extreme = gradients[extreme_outliers]
    labels_extreme = labels[extreme_outliers]
    colors_extreme = colors[extreme_outliers]
    imgIndx = imgIndx[extreme_outliers]

    # Create Dataframe for easy use
    df = pd.DataFrame({'Index':imgIndx,'Activations': activations_extreme, 'Gradients': gradients_extreme, 'Labels': labels_extreme})
    
    if selected_class > -1:
        colors_extreme = colors_extreme[df['Labels'] == selected_class]
        df = df[df['Labels'] == selected_class]
        
    if quadrant == 1:
        colors_extreme = colors_extreme[df['Gradients'] > 0]
        df = df[df['Gradients'] > 0]
        colors_extreme = colors_extreme[df['Activations'] > 0]
        df = df[df['Activations'] > 0]
    elif quadrant == 2:
        colors_extreme = colors_extreme[df['Gradients'] > 0]
        df = df[df['Gradients'] > 0]
        colors_extreme = colors_extreme[df['Activations'] < 0]
        df = df[df['Activations'] < 0]
        
    elif quadrant == 3:
        colors_extreme = colors_extreme[df['Gradients'] < 0]
        df = df[df['Gradients'] < 0]
        colors_extreme = colors_extreme[df['Activations'] < 0]
        df = df[df['Activations'] < 0]
        
    elif quadrant == 4:
        colors_extreme = colors_extreme[df['Gradients'] < 0]
        df = df[df['Gradients'] < 0]
        colors_extreme = colors_extreme[df['Activations'] > 0]
        df = df[df['Activations'] > 0]
        
    # Generate scatter plot with extreme outliers
    fig, ax = plt.subplots(figsize=(15, 10))
    scatter = ax.scatter(df.loc[:, ['Activations']], df.loc[:, ['Gradients']], c=colors_extreme, cmap=color_map, s=15)
    ax.set_xlabel('Activation')
    ax.set_ylabel('Negative of Gradient')
    ax.set_title(f'Extreme Outliers Scatter Plot for {layer_name} - Node {node_index}')

    # Create a colorbar
    cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=color_map, norm=plt.Normalize(0, 9)), ax=ax, ticks=np.arange(0, 10))
    cbar.set_label('Digit')

    ax.grid(True)
    plt.show()


# Example usage:
plot_extreme_outliers_quad(explainable_net, 'fc3', 3, std_dev_threshold=1, quadrant= 4, selected_class=2)  # Adjust std_dev_threshold as needed



In [ ]:
import pandas as pd

def extreme_outliers_quad(explainable_network, layer_name, node_index, quadrant, std_dev_threshold=3, selected_class=-1):
    activations = []
    gradients = []
    labels = []
    imgIndx = []

    # Retrieve activations, gradients, and check for correct predictions
    for input_index, (expected, predicted) in enumerate(zip(explainable_network.expected_outputs, explainable_network.predicted_outputs)):
        #if expected == predicted: do we want to filter this?
        activation = explainable_network.get_activations(input_index)
        gradient = explainable_network.get_gradients(input_index)

        if activation is not None and gradient is not None:
            activations.append(activation[layer_name][node_index])
            gradients.append(-gradient[layer_name][node_index])  # Negative of the gradient
            labels.append(expected)
            imgIndx.append(input_index)
            

    # Convert lists to numpy arrays for numerical operations
    activations = np.array(activations)
    gradients = np.array(gradients)
    labels = np.array(labels)
    imgIndx = np.array(imgIndx)


    # Calculate mean and standard deviation for activations and gradients
    activation_mean = np.mean(activations)
    activation_std = np.std(activations)
    gradient_mean = np.mean(gradients)
    gradient_std = np.std(gradients)

    # Apply filtering for points that are more than std_dev_threshold away in both dimensions
    extreme_outliers = (np.abs(activations - activation_mean) > std_dev_threshold * activation_std) & \
                       (np.abs(gradients - gradient_mean) > std_dev_threshold * gradient_std)

    # Filter activations, gradients, and colors based on the extreme outlier condition
    activations_extreme = activations[extreme_outliers]
    gradients_extreme = gradients[extreme_outliers]
    labels_extreme = labels[extreme_outliers]
    imgIndx = imgIndx[extreme_outliers]

    # Create Dataframe for easy use
    df = pd.DataFrame({'Index':imgIndx,'Activations': activations_extreme, 'Gradients': gradients_extreme, 'Labels': labels_extreme})
    
    if selected_class > -1:
        df = df[df['Labels'] == selected_class]
        
    if quadrant == 1:
        df = df[df['Activations'] > 0]
        df = df[df['Gradients'] > 0]
        
    elif quadrant == 2:
        df = df[df['Activations'] < 0]
        df = df[df['Gradients'] > 0]
        
    elif quadrant == 3:
        df = df[df['Activations'] < 0]
        df = df[df['Gradients'] < 0]
        
    elif quadrant == 4:
        df = df[df['Activations'] > 0]
        df = df[df['Gradients'] < 0]
    
    return df['Index'].values


# Example usage:
extreme_outliers_quad(explainable_net, 'fc3', 3, std_dev_threshold=1, quadrant= 2)  # Adjust std_dev_threshold as needed

